In [ ]:
from nbdev import *
%nbdev_default_export migrate2magic

Cells will be exported to nbdev.migrate2magic,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Migrate from comment flags to magic flags

> Console command to migrate comment flags to magic flags

In [ ]:
%nbdev_export
from nbdev.imports import *
from nbdev.export import read_nb
from fastscript import call_parse,Param
import re,nbformat

To use the command created by this notebook, you must be in one of the subfolders of your project: it will search for `settings.ini` recursively in the parent directory but need to access it to be able to work. 
- `nbdev_migrate2magic` updates comment flags to magic flags in notebooks

## Migrating notebooks

Run `nbdev_migrate2magic` from the command line to update code cells in notebooks that use comment flags like

```python
# default_exp migrate2magic
#export special.module
```

to use magic flags

```python
%nbdev_default_export migrate2magic
%nbdev_export special.module
```

To make the magic flags work, `nbdev_migrate2magic` adds a new code cell to the top of the notebook

```python
from nbdev import *
```

Note: we don't add this new cell if we don't find any comment flags to replace.

After running `nbdev_migrate2magic`, you'll see
- migrated notebooks in `nbs_path`
- unmodified notebooks in `.nbdev_bck_0`.

### Hiding the `from nbdev import *` cell

nbdev does not treat `from nbdev import *` as special but this cell can be hidden from the docs by combining it with `%nbdev_default_export`. e.g. 
```python
from nbdev import *
%nbdev_default_export migrate2magic
```
this works because nbdev will hide any cell containing the `%nbdev_default_export` flag.

In [ ]:
%nbdev_export_internal
def _code_patterns_and_replace_fns():
    "return a list of pattern/function tuples that can migrate flags used in code cells"
    patterns_and_replace_fns = []

    def _replace_fn(magic, m):
        "return a magic flag for a comment flag matched in `m`"
        return f'%{magic}' if m.group(1) is None else f'%{magic} {m.group(1).strip()}'

    def _add_pattern_and_replace_fn(comment_flag, magic_flag):
        "add a pattern/function tuple to go from comment to magic flag"
        pattern = re.compile(rf"""
# Matches a comment flag line (e.g. #exporti) and catches parameter in group 1:
^              # beginning of line (since re.MULTILINE is passed)
\s*            # any number of whitespace
\#\s*          # "#", then any number of whitespace
{comment_flag} #
([ \t]+\S+)?   # catch a group, with leading spaces and/or tabs, followed by any non-whitespace chars
\s*            # any number of whitespace
$              # end of line (since re.MULTILINE is passed)
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)
        # note: fn has to be single arg so we can use it in `pattern.sub` calls later
        patterns_and_replace_fns.append((pattern, partial(_replace_fn, magic_flag)))

    _add_pattern_and_replace_fn('default_exp', 'nbdev_default_export')
    _add_pattern_and_replace_fn('exports', 'nbdev_export_and_show')
    _add_pattern_and_replace_fn('exporti', 'nbdev_export_internal')
    _add_pattern_and_replace_fn('export', 'nbdev_export')
    _add_pattern_and_replace_fn('hide_input', 'nbdev_hide_input')
    _add_pattern_and_replace_fn('hide_output', 'nbdev_hide_output')
    _add_pattern_and_replace_fn('hide', 'nbdev_hide')
    _add_pattern_and_replace_fn('default_cls_lvl', 'nbdev_default_class_level')
    _add_pattern_and_replace_fn('collapse[_-]output', 'nbdev_collapse_output')
    _add_pattern_and_replace_fn('collapse[_-]show', 'nbdev_collapse_input open')
    _add_pattern_and_replace_fn('collapse[_-]hide', 'nbdev_collapse_input')
    _add_pattern_and_replace_fn('collapse', 'nbdev_collapse_input')
    for flag in Config().get('tst_flags', '').split('|'):
        _add_pattern_and_replace_fn(f'all_{flag}', f'nbdev_{flag}_test all')
        _add_pattern_and_replace_fn(flag, f'nbdev_{flag}_test')
    return patterns_and_replace_fns

In [ ]:
%nbdev_export_internal
class CellMigrator():
    """Can migrate a cell using `patterns_and_replace_fns`.
    Keeps track of the number of cells updated in `upd_count`"""
    def __init__(self, patterns_and_replace_fns):
        self.patterns_and_replace_fns,self.upd_count=patterns_and_replace_fns,0
    def __call__(self, cell):
        for pattern, replace_fn in self.patterns_and_replace_fns:
            source=cell.source
            cell.source=pattern.sub(replace_fn, source)
            if source!=cell.source: self.upd_count+=1

In [ ]:
%nbdev_export_internal
def _migrate2magic(nb, update_md=False):
    "Migrate a single notebook"
    code_cell_migrator=CellMigrator(_code_patterns_and_replace_fns())
    [code_cell_migrator(cell) for cell in nb.cells if cell.cell_type=='code']
    if code_cell_migrator.upd_count!=0:
        nb.cells.insert(0, nbformat.v4.new_code_cell('from nbdev import *'))
    NotebookNotary().sign(nb)
    return nb

In [ ]:
%nbdev_hide
def remove_line(starting_with, from_string):
    result=[]
    for line in from_string.split('\n'):
        if not line.strip().startswith(starting_with):
            result.append(line)
    return '\n'.join(result)

test_eq('x\nb\n%magic\n123\n %magic', remove_line('#', 'x\nb\n%magic\n#comment\n123\n %magic\n # comment'))
test_eq('x\nb\n123', remove_line('%', 'x\nb\n%magic\n123\n %magic'))

In [ ]:
%nbdev_hide
def remove_comments_and_magics(string):
    return remove_line('#', remove_line('%', string))

test_eq('x\nb\n123', remove_comments_and_magics('x\nb\n%magic\n#comment\n123\n %magic\n # comment'))

In [ ]:
%nbdev_hide
def test_migrate2magic(fname):
    "check that nothing other that comments and magics in code cells have been changed"
    nb=read_nb(fname)
    nb_migrated=_migrate2magic(read_nb(fname))
    test_eq(len(nb.cells)+1, len(nb_migrated.cells))
    test_eq('from nbdev import *', nb_migrated.cells[0].source)
    for i in range(len(nb.cells)):
        cell, cell_migrated=nb.cells[i], nb_migrated.cells[i+1]
        if cell.cell_type=='code':
            cell.source=remove_comments_and_magics(cell.source)
            cell_migrated.source=remove_comments_and_magics(cell_migrated.source)
        test_eq(cell, cell_migrated)
        
test_migrate2magic('../test/00_export.ipynb')
test_migrate2magic('../test/07_clean.ipynb')

def test_migrate2magic_noop(fname):
    "check that nothing is changed if there are no comment flags in a notebook"
    nb=read_nb(fname)
    nb_migrated=_migrate2magic(read_nb(fname))
    test_eq(nb, nb_migrated)
        
test_migrate2magic_noop('99_search.ipynb')

In [ ]:
%nbdev_hide
sources=['#export aaa\nimport io,sys,json,glob\n#collapse-OUTPUT\nfrom fastscript ...',
           '%nbdev_export aaa\nimport io,sys,json,glob\n%nbdev_collapse_output\nfrom fastscript ...',
           '#EXPORT\n # collapse\nimport io,sys,json,glob',
           '%nbdev_export\n%nbdev_collapse_input\nimport io,sys,json,glob',
           '#exportS\nimport io,sys,json,glob\n#colLApse_show',
           '%nbdev_export_and_show\nimport io,sys,json,glob\n%nbdev_collapse_input open',
           ' # collapse-show \n#exporti\nimport io,sys,json,glob',
           '%nbdev_collapse_input open\n%nbdev_export_internal\nimport io,sys,json,glob',
           '#export\t\tspecial.module  \nimport io,sys,json,glob',
           '%nbdev_export special.module\nimport io,sys,json,glob',
           '#exports special.module\nimport io,sys,json,glob',
           '%nbdev_export_and_show special.module\nimport io,sys,json,glob',
           '#EXPORTs   special.module\nimport io,sys,json,glob',
           '%nbdev_export_and_show special.module\nimport io,sys,json,glob',
           '#exportI \t \tspecial.module\n# collapse_hide  \nimport io,sys,json,glob',
           '%nbdev_export_internal special.module\n%nbdev_collapse_input\nimport io,sys,json,glob',
           '# export \nimport io,sys,json,glob',
           '%nbdev_export\nimport io,sys,json,glob',
           '# exports \nimport io,sys,json,glob',
           '%nbdev_export_and_show\nimport io,sys,json,glob',
           '# exporti \nimport io,sys,json,glob',
           '%nbdev_export_internal\nimport io,sys,json,glob',
           ' # export\nimport io,sys,json,glob',
           '%nbdev_export\nimport io,sys,json,glob',
           ' #  Collapse-Hide \n # EXPORTS\nimport io,sys,json,glob',
           '%nbdev_collapse_input\n%nbdev_export_and_show\nimport io,sys,json,glob',
           ' # exporti\nimport io,sys,json,glob',
           '%nbdev_export_internal\nimport io,sys,json,glob',
           'import io,sys,json,glob\n#export aaa\nfrom fastscript import call_pars...',
           'import io,sys,json,glob\n%nbdev_export aaa\nfrom fastscript import call_pars...',
           '#COLLAPSE_OUTPUT\nprint("lotsofoutput")',
           '%nbdev_collapse_output\nprint("lotsofoutput")']
nb=nbformat.v4.new_notebook()
nb_expected=nbformat.v4.new_notebook()
nb_expected.cells.append(nbformat.v4.new_code_cell('from nbdev import *'))
for cells, source in zip([nb.cells,nb_expected.cells]*(len(sources)//2), sources):
    cells.append(nbformat.v4.new_code_cell(source))
nb_migrated=_migrate2magic(nb)
test_eq(nb_expected, nb_migrated)

`nbdev_migrate2magic` copies all notebooks in `nbs_path` to `.nbdev_bck_0` (or `.nbdev_bck_1` if `_0` already exsts ...)

`nbdev_migrate2magic` then reads all notebooks in `nbs_path` and migrates them in-place.

In [ ]:
%nbdev_export
@call_parse
def nbdev_migrate2magic():
    """Update all notebooks in `nbs_path` to use magic flags."""
    config=Config()
    bck_path=config.config_file.parent/'.nbdev_bck_0'
    i=0
    while bck_path.exists():
        i+=1
        bck_path=config.config_file.parent/f'.nbdev_bck_{i}'
    bck_path.mkdir()
    for fname in config.nbs_path.glob('*.ipynb'):
        shutil.copy2(fname, bck_path)
    print('Copied nbs in', config.nbs_path, 'to', bck_path)
    for fname in config.nbs_path.glob('*.ipynb'):
        print('Migrating', fname)
        nbformat.write(_migrate2magic(read_nb(fname)), str(fname), version=4)

## Export

In [ ]:
%nbdev_hide
from nbdev.export import *
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 08_flags.ipynb.
Converted 09_migrate2magic.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted tutorial.ipynb.
